# Get rows with only latest transaction

- Use the zillow dataset
- Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.
- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.

- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.

- Only include properties that include a latitude and longitude value.


## Solution
1. GroupBy parcelid
2. Take the max transaction date from the groupby object (a Series)
3. Turn the max transaction date by parcel Series into a dataframe
4. Merge/join the max transaction date series dataframe with the original dataframe to filter out older transaction dates

In [1]:
import pandas as pd
from wrangle import get_db_url
url = get_db_url('zillow')

In [2]:
sql = """
SELECT *
FROM properties_2017
JOIN predictions_2017 using(parcelid)
LEFT JOIN airconditioningtype air USING (airconditioningtypeid) 
LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid) 
LEFT JOIN buildingclasstype build USING (buildingclasstypeid) 
LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid) 
LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid) 
LEFT JOIN storytype story USING (storytypeid) 
LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid) 
WHERE latitude IS NOT NULL AND longitude IS NOT NULL
AND transactiondate < '2018-01-01'
"""

df = pd.read_sql(sql, url)
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,1727539,NaN,...,0,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1387261,NaN,...,1,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,11677,NaN,...,2,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,2288172,NaN,...,3,-0.103410,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10887214,1970746,NaN,...,4,0.006940,2017-01-01,Central,None,None,Central,Condominium,None,None


In [3]:
df.parcelid.duplicated().sum()

198

In [4]:
n_duplicates = df.parcelid.duplicated().sum()
n_duplicates

198

In [7]:
df.transactiondate.dtype

dtype('O')

In [8]:
# Convert transaction date into proper datetime format
df.transactiondate = pd.to_datetime(df.transactiondate)

In [9]:
df.transactiondate

0       2017-01-01
1       2017-01-01
2       2017-01-01
3       2017-01-01
4       2017-01-01
           ...    
77574   2017-09-20
77575   2017-09-20
77576   2017-09-21
77577   2017-09-21
77578   2017-09-25
Name: transactiondate, Length: 77579, dtype: datetime64[ns]

In [10]:
df.transactiondate.dtype

dtype('<M8[ns]')

In [19]:
df.groupby('parcelid').transactiondate.max()

parcelid
10711855    2017-07-07
10711877    2017-08-29
10711888    2017-04-04
10711910    2017-03-17
10711923    2017-03-24
               ...    
167686999   2017-02-28
167687739   2017-03-03
167687839   2017-05-31
167688532   2017-02-03
167689317   2017-03-14
Name: transactiondate, Length: 77381, dtype: datetime64[ns]

In [20]:
# Obtain the maximimum transaction date by each parcelid
# For parcelids that show once, this returns the only transactiondate
# For parcelids showing multiple times, this returns the max transactiondate
max_transaction_date_by_parcel = df.groupby('parcelid').transactiondate.max()
max_transaction_date_by_parcel

parcelid
10711855    2017-07-07
10711877    2017-08-29
10711888    2017-04-04
10711910    2017-03-17
10711923    2017-03-24
               ...    
167686999   2017-02-28
167687739   2017-03-03
167687839   2017-05-31
167688532   2017-02-03
167689317   2017-03-14
Name: transactiondate, Length: 77381, dtype: datetime64[ns]

In [21]:
pd.DataFrame(max_transaction_date_by_parcel)

,transactiondate
parcelid,
10711855,2017-07-07
10711877,2017-08-29
10711888,2017-04-04
10711910,2017-03-17
10711923,2017-03-24
...,...
167686999,2017-02-28
167687739,2017-03-03
167687839,2017-05-31


In [22]:
# Convert the above Series into a Dataframe
max_transaction_date_by_parcel = pd.DataFrame(max_transaction_date_by_parcel)
max_transaction_date_by_parcel['parcelid'] = max_transaction_date_by_parcel.index
max_transaction_date_by_parcel = max_transaction_date_by_parcel.reset_index(drop=True)
max_transaction_date_by_parcel

,transactiondate,parcelid
0,2017-07-07,10711855
1,2017-08-29,10711877
2,2017-04-04,10711888
3,2017-03-17,10711910
4,2017-03-24,10711923
...,...,...
77376,2017-02-28,167686999
77377,2017-03-03,167687739
77378,2017-05-31,167687839
77379,2017-02-03,167688532


In [23]:
# Merge/join on both the transaction date AND parcel id in order to filter out the older transactiondates by parcel
# The merge/join is where the "magic" happens of removing rows with duplicate parcelids w/ less recent transactiondates
output = df.merge(max_transaction_date_by_parcel, on=["transactiondate", "parcelid"])
output

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,id,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,1727539,NaN,...,0,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1387261,NaN,...,1,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,11677,NaN,...,2,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,2288172,NaN,...,3,-0.103410,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10887214,1970746,NaN,...,4,0.006940,2017-01-01,Central,None,None,Central,Condominium,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77376,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10833991,2864704,NaN,...,77608,-0.002245,2017-09-20,Central,None,None,Central,Condominium,None,None
77377,NaN,NaN,261.0,2.0,NaN,NaN,NaN,11000655,673515,NaN,...,77609,0.020615,2017-09-20,None,None,None,Central,Single Family Residential,None,None
77378,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17239384,2968375,NaN,...,77610,0.013209,2017-09-21,None,None,None,None,Single Family Residential,None,None
77379,NaN,NaN,261.0,2.0,NaN,NaN,1.0,12773139,1843709,NaN,...,77611,0.037129,2017-09-21,Central,None,None,Central,Single Family Residential,None,None


In [24]:
assert n_duplicates == (df.shape[0] - output.shape[0])
print("Number of duplicates is equal to the length of the original dataframe minus the length of the final dataframe")

Number of duplicates is equal to the length of the original dataframe minus the length of the final dataframe


In [25]:
output.parcelid.duplicated().sum()

0

In [26]:
assert output.parcelid.duplicated().sum() == 0
print("No duplicate parcelid values in the final dataframe")

No duplicate parcelid values in the final dataframe
